# [1단계] 새 노트북 환경 설정

## 1-1. 필요한 라이브러리 설치 및 임포트

In [1]:
# transformers: Qwen2 토크나이저를 불러와 템플릿을 적용하는 데 필요
# datasets, pandas, huggingface_hub: 데이터셋을 다루고 업로드하는 데 필수
!pip install -qU transformers datasets "pandas==2.2.2" huggingface_hub

from datasets import load_dataset, concatenate_datasets, DatasetDict
from transformers import AutoTokenizer
from huggingface_hub import login
from google.colab import userdata
import pandas as pd

print("✅ 라이브러리 준비 완료!")

✅ 라이브러리 준비 완료!


## 1-2. Hugging Face 로그인

In [2]:
# Colab '비밀' 탭에 저장된 토큰으로 로그인하여 비공개 데이터셋에 접근할 권한을 얻는다.
try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("✅ Hugging Face 로그인 성공!")
except Exception as e:
    print(f"🚨 Hugging Face 로그인 실패: {e}")

✅ Hugging Face 로그인 성공!


# [2단계] 최종 학습용 데이터셋 2종 로드

In [3]:
print("--- 최종 학습에 사용할 데이터셋 2종을 로드합니다 ---")

try:
    # Colab 비밀 저장소에서 허깅페이스 사용자 이름을 가져온다.
    hf_username = userdata.get('HF_USERNAME')

    # 1. 템플릿이 적용된 30k+ 베이스 데이터셋 로드
    # 이 데이터셋은 이미 'train'과 'test'로 나뉘어 있고, Qwen2 템플릿이 적용되어 있다.
    base_dataset_repo = f"{hf_username}/finance_kb_common_sense_mix_dataset_30k"
    base_dataset_dict = load_dataset(base_dataset_repo)
    print(f"✅ 1. 베이스 데이터셋 '{base_dataset_repo}' 로드 완료!")
    print(base_dataset_dict)

    # 2. 파일명이 정리된 KB 특화 Q&A 데이터셋 로드
    # 이 데이터셋도 'train'과 'test'로 나뉘어 있지만, 아직 Qwen2 템플릿이 적용되지 않은 상태.
    kb_dataset_repo = f"{hf_username}/KB-sLLM-QA-Dataset-Final-Split"
    kb_dataset_dict = load_dataset(kb_dataset_repo)
    print(f"\n✅ 2. KB 특화 데이터셋 '{kb_dataset_repo}' 로드 완료!")
    print(kb_dataset_dict)

except Exception as e:
    print(f"🚨 데이터셋 로드 중 오류가 발생했습니다: {e}")

--- 최종 학습에 사용할 데이터셋 2종을 로드합니다 ---


README.md:   0%|          | 0.00/399 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/34.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29599 [00:00<?, ? examples/s]

...

train-00000-of-00001.parquet:   0%|          | 0.00/197k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1007 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/150 [00:00<?, ? examples/s]


✅ 2. KB 특화 데이터셋 'rucipheryn/KB-sLLM-QA-Dataset-Final-Split' 로드 완료!
DatasetDict({
    train: Dataset({
        features: ['question', 'answer_B', 'source_pdf'],
        num_rows: 1007
    })
    test: Dataset({
        features: ['question', 'answer_B', 'source_pdf'],
        num_rows: 150
    })
})


# [3단계] KB 특화 데이터셋 형식 변환

## 3-1. Qwen2 토크나이저 로드

In [4]:
# 템플릿 적용을 위해 Qwen2 모델의 토크나이저를 불러온다.
model_id = "Qwen/Qwen2.5-14B-Instruct"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    print(f"✅ '{model_id}' 토크나이저 로드 성공!")
except Exception as e:
    print(f"🚨 토크나이저 로드 실패: {e}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ 'Qwen/Qwen2.5-14B-Instruct' 토크나이저 로드 성공!


## 3-2. KB 데이터셋을 표준 대화 형식으로 변환하는 함수

In [5]:
def format_kb_qa_to_conversations(example):
    """
    'question'과 'answer_B' 컬럼을 가진 KB 데이터셋을
    [{'role': 'user', ...}, {'role': 'assistant', ...}] 형태로 변환한다.
    """
    return {
        "conversations": [
            {"role": "user", "content": example["question"]},
            {"role": "assistant", "content": example["answer_B"]}
        ]
    }

## 3-3. 챗 템플릿을 적용하는 함수

In [6]:
def apply_chat_template(example):
    """
    'conversations' 리스트를 토크나이저를 사용해 단일 'text'로 변환한다.
    """
    return {"text": tokenizer.apply_chat_template(
        example["conversations"], tokenize=False, add_generation_prompt=False
    )}

## 3-4. KB 데이터셋에 함수들을 순차적으로 적용

In [7]:
print("\n--- KB 특화 데이터셋에 템플릿 적용을 시작합니다 ---")

# .map()을 연달아 사용하여 두 가지 변환을 한 번에 처리.
kb_dataset_templated_dict = kb_dataset_dict.map(format_kb_qa_to_conversations) \
                                           .map(apply_chat_template, remove_columns=['question', 'answer_B', 'source_pdf', 'conversations'])

print("✅ KB 특화 데이터셋 형식 변환 완료!")
print("\n--- 변환 후 KB 데이터셋 정보 ---")
print(kb_dataset_templated_dict)
print("\n--- 변환 후 샘플 데이터 ---")
print(kb_dataset_templated_dict['train'][0]['text'])


--- KB 특화 데이터셋에 템플릿 적용을 시작합니다 ---


Map:   0%|          | 0/1007 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/1007 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

✅ KB 특화 데이터셋 형식 변환 완료!

--- 변환 후 KB 데이터셋 정보 ---
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1007
    })
    test: Dataset({
        features: ['text'],
        num_rows: 150
    })
})

--- 변환 후 샘플 데이터 ---
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
원리금균등 분할상환과 원금균등 분할상환의 차이점은 무엇인가요?<|im_end|>
<|im_start|>assistant
원리금균등 분할상환은 매월 이자지급일에 동일한 할부금을 상환하는 방식이며, 원금균등 분할상환은 매월 이자지급일에 이자와 동일한 할부 원금을 상환하는 방식입니다.<|im_end|>



# [4단계] 데이터셋 통합 및 최종 업로드

## 4-1. Train / Test 스플릿별로 데이터셋 통합

In [8]:
# concatenate_datasets 함수를 사용해 여러 데이터셋을 하나로 합친다.
# train 스플릿 합치기
combined_train_dataset = concatenate_datasets([
    base_dataset_dict['train'],
    kb_dataset_templated_dict['train']
])

# test 스플릿 합치기
combined_test_dataset = concatenate_datasets([
    base_dataset_dict['test'],
    kb_dataset_templated_dict['test']
])

# 합쳐진 데이터셋들을 최종적으로 한번 더 섞어준다 (재현성을 위해 seed 고정).
combined_train_dataset = combined_train_dataset.shuffle(seed=42)
combined_test_dataset = combined_test_dataset.shuffle(seed=42)

## 4-2. 최종 DatasetDict 생성

In [9]:
# 합쳐진 train/test 데이터셋을 하나의 DatasetDict로 묶는다.
final_combined_dataset = DatasetDict({
    'train': combined_train_dataset,
    'test': combined_test_dataset
})

print("--- 최종 데이터셋 통합 완료 ---")
print(final_combined_dataset)

--- 최종 데이터셋 통합 완료 ---
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 30606
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1708
    })
})


## 4-3. 최종 통합 데이터셋을 Hugging Face Hub에 업로드

In [10]:
try:
    hf_username = userdata.get('HF_USERNAME')
    # 최종 데이터셋의 이름을 지정한다.
    repo_name = f"{hf_username}/combined-dataset-30K-final"

    print(f"\n🚀 최종 통합 데이터셋을 Hugging Face Hub에 업로드합니다...")
    print(f"Repository: {repo_name}")

    # DatasetDict 형태로 업로드하면 'train'과 'test' split이 함께 저장된다.
    final_combined_dataset.push_to_hub(
        repo_id=repo_name,
        private=True, # 비공개로 설정
        commit_message="Combine base dataset and KB-specific dataset"
    )

    print("\n🎉 최종 데이터셋 업로드 성공!")
    print("이 데이터셋을 사용하여 파인튜닝을 시작할 수 있습니다.")
    print(f"URL: https://huggingface.co/datasets/{repo_name}")

except Exception as e:
    print(f"🚨 데이터셋 업로드 중 오류가 발생했습니다: {e}")


🚀 최종 통합 데이터셋을 Hugging Face Hub에 업로드합니다...
Repository: rucipheryn/combined-dataset-30K-final


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]


🎉 최종 데이터셋 업로드 성공!
이 데이터셋을 사용하여 파인튜닝을 시작할 수 있습니다.
URL: https://huggingface.co/datasets/rucipheryn/combined-dataset-30K-final
